In [2]:
from vedo import dataurl, Mesh, Plotter, Volume, settings, LinearTransform, Box, Line, Sphere
import numpy as np
import pandas as pd
from normalize import normalize_shape, get_center_of_mass,normalize_vertices,normalize_scale
import os
from pathlib import Path
import matplotlib.pyplot as plt
settings.default_backend = 'k3d'
np.set_printoptions(suppress=True)


In [3]:
def get_eigenvectors(mesh: Mesh):
    """
    Returns eigenvectors and eigenvalues of a mesh
    ----------------------------
    Args:
        Vedo Mesh
    Returns:
        Tuple of (eigenvalues, eigenvectors)
    """
    A = np.zeros((3, len(mesh.vertices)))
    A[0] = mesh.vertices[:, 0]
    A[1] = mesh.vertices[:, 1]
    A[2] = mesh.vertices[:, 2]

    # compute the covariance matrix for A 
    # see the documentation at 
    # https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html
    # this function expects that each row of A represents a variable, 
    # and each column a single observation of all those variables
    A_cov = np.cov(A)  # 3x3 matrix

    # computes the eigenvalues and eigenvectors for the 
    # covariance matrix. See documentation at  
    # https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html 
    eigenvalues, eigenvectors = np.linalg.eig(A_cov)

    print("==> eigenvalues for (x, y, z)")
    print(eigenvalues)
    print("\n==> eigenvectors")
    print(eigenvectors)
    A_cov = np.cov(np.transpose(mesh.vertices))  # 3x3 matrix
    # computes the eigenvalues and eigenvectors for the
    # covariance matrix. See documentation at
    # https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html
    eigenvalues, eigenvectors = np.linalg.eig(A_cov)
    # print(eigenvectors,eigenvalues) TODO: remove?
    return eigenvectors, eigenvalues

In [4]:
unorm = normalize_shape(Sphere(r=1, res=500, quads=False, c='red', alpha=1.0)).c("Black")
eigenvectors, eigenvals = get_eigenvectors(unorm)
#print(mesh_v4)
print(unorm.vertices[:, 2])
print(f"Eccentricity: {max(eigenvals)/min(eigenvals)}")


==> eigenvalues for (x, y, z)
[0.08303283 0.08348733 0.0835591 ]

==> eigenvectors
[[-1.         -0.000002    0.00000066]
 [-0.000002    1.          0.00003992]
 [ 0.00000066 -0.00003992  1.        ]]
[0.11350152 0.12970169 0.13447422 ... 0.39957035 0.42497876 0.44312367]
Eccentricity: 1.006338105363994


In [7]:
unorm2 = normalize_shape(Sphere(r=1, res=10, quads=False, c='red', alpha=1.0)).c("Black")
eigenvectors, eigenvals = get_eigenvectors(unorm2)
#print(mesh_v4)
print(unorm2.vertices[:, 2])
print(f"Eccentricity: {max(eigenvals)/min(eigenvals)}")

==> eigenvalues for (x, y, z)
[0.07464437 0.07464437 0.09340831]

==> eigenvectors
[[ 1.         -0.04456781 -0.        ]
 [ 0.          0.99900636 -0.        ]
 [ 0.         -0.         -1.        ]]
[ 0.5        -0.5         0.4698463  ... -0.39387524 -0.39387524
 -0.40472823]
Eccentricity: 1.2513779411707147


In [9]:
unorm2.wireframe(True)
unorm.wireframe(True)
unorm2.vertices+=[0,0,1]



In [22]:
print((unorm.vertices[:, 2].std()))


0.31759736


In [10]:
#Rich Display

z_axis = Line([0,0,0], [0,0,2], lw=3).c("Blue")
y_axis = Line([0,0,0], [0,2,0], lw=3).c("Green")
x_axis = Line([0,0,0], [2,0,0], lw=3).c("Red")

plot = Plotter(bg='white')
#plot.show(mesh_v3,mesh2,x_axis,y_axis,z_axis,unit_box,eig1,eig2,eig3)
plot.show(unorm,unorm2,x_axis,y_axis,z_axis,eig1,eig2,eig3)

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [5]:
print(mesh_v2.filename)

../shapes/FloorLamp/m619.obj


In [34]:
print(mesh)
print(mesh_v2)

vedo.mesh.Mesh at (0x196a7122ea0)                                          
name          : Mesh
file name     : ../shapes\FloorLamp\m619.obj
elements      : vertices=361 polygons=696 lines=0
position      : (0, 0, 0)
scaling       : (1.00000, 1.00000, 1.00000)
size          : average=0.133367, diagonal=0.973164
center of mass: (0.0996223, 0.828652, 0.0996225)
bounds        : x=(0.0250, 0.174), y=(0.0250, 0.975), z=(0.0250, 0.174)
pointdata     : "Normals" (float32), dim=3
celldata      : "GroupIds" (float32), dim=1, range=(0, 0)
vedo.mesh.Mesh at (0x196a7123ca0)                                          
name          : Mesh
file name     : ../shapes\FloorLamp\m619.obj
color         : red, rgb=(0.996, 0.118, 0.122), alpha=1.0
elements      : vertices=18,025 polygons=36,024 lines=0
position      : (0, 0, 0)
scaling       : (1.00000, 1.00000, 1.00000)
size          : average=0.198285, diagonal=0.973164
center of mass: (0.0996227, 0.440077, 0.0996249)
bounds        : x=(0.0250, 0.174), y=

In [58]:
def get_vertex_dist(mesh:Mesh):
    #As the ratio of the smallest cell against largest cell
    minar = np.inf
    maxar = 0
    for idx in range(0,len(mesh.vertices)):
        ar = 0
        for cp in mesh.connected_vertices(idx):
            ar+= np.abs(np.linalg.norm(mesh.vertices[idx] - mesh.vertices[cp]))
        

        if ar!=0 and ar<minar:
            minar=ar
        elif ar!=0 and ar>maxar:
            maxar=ar
        #print(ar,minar,maxar)

    return maxar/minar

print(get_vertex_dist(mesh))
get_vertex_dist(mesh_v4)


    

25.951030928079508


18.473870082480552